## Binary Files
- We've been reading and writing plaintext files.  Only basic data types such as integers, floats, strings, lists, and dictionaries can be written to a file as plaintext. Other objects like Regex and file objects, cannot be represented as text.  These must be saved as binary files.

---

### Pickle and Shelve

1. **Pickle**--encode Python data into a binary file.  Can take any Python object, pickle it, and store it as a binary file for later consumption (mmm...pickles).    Pickled data can only be read by the Python program.  Functions found in `pickle` module, however, we often don't directly interact with the `pickle` module.  Instead we interact with the `shelve` module.
1. **Shelve**--built on top of `pickle`.  Functions found in `shelve` module.  Creates a shelf file similar to a dictionary.  Has keys and values.  Key is a text string, while value is a pickled Python object that we want to place in a shelf (file) and store (mmm...shelf pickles).  We can retrieve the pickled binary file by entering a text key.  A shelf file can be a good way to store Python data long term if it can not be stored as plaintext.

Code | Use
--- | ---
`shelve` | Module
`shelve.open()` | Create shelf object, which is like a dictionary. **CAUTION!** only accepts paths written as strings.  Error if path object entered.
`<SHELF_OBJECT>['<KEY>'] = <PYTHON_OBJECT>` | Grammar for storing Python object (value) to text key in shelf file
`<SHELF_OBJECT>.keys()` | List keys
`del <SHELF_OBJECT>[<KEY>]` | Delete key: value pair
`.close` | Shelf object method.  Close shelf file.
`with shelve.open() as <SHELF_OBJECT>:` | *Context Manager* combines open and close.  It automatically closes the file when we are done with it.  It knows we are done when we de-indent.  It also closes a file correctly if there is an exception at some point in the file handling.  Using the context manager is recommended.

---

**EXAMPLES**

In [1]:
import shelve

In [2]:
l_breakfast = ['spam', 'beans', 'spam']  # list is Python object that will be encoded as a binary file

with shelve.open('./output/my_shelf_file') as shelf_object:
    shelf_object['l_breakfast'] = l_breakfast  # 'l_breakfast' is text key

In [3]:
with shelve.open('./output/my_shelf_file') as shelf_object:
    print(type(shelf_object))
    print(list(shelf_object.keys()))
    print(shelf_object['l_breakfast'])

<class 'shelve.DbfilenameShelf'>
['l_breakfast']
['spam', 'beans', 'spam']


---

### RDBMS
- We can also read and write data with a DBMS file
- **Database management system (DBMS)**--computer program used to manage data.  DBMS can be non-relational in nature (called NoSQL), but the vast majority are relational databases that use SQL.   Theses are called relational DBMS, or RDBMS. 
- **Relational Database**--collection of tables that are *related* to each other by data common to each. This is the most common way to store data.
- Databases are fundamentally CRUD:
    1. Create data
    1. Read (query) data
    1. Update data
    1. Delete data
- **SQL**--Structured Query Language.  Communicate CRUD with RDBMS.
- We can CRUD with a Python script by using a combination of Python commands, SQL commands, and SQLite
- **SQLite**--RDBMS. SQLite stores data in one file on a hard drive.  Most RDBMS store a batch of separate files on a network.  SQLite is super simple and lightweight.  The most used data base in the world and embedded into many apps on phones, cars, etc.  It is actually built into Python, so we do not have to install any additional software in order to use it! The only down side is that SQLite is not great for large databases.

Code | Use
--- | ---
`sqlite3` | Module
`sqlite3.connect()` | Create connection object representing the database.  Parameter is database file path.  If the file does not exist it will be created.  
`.cursor()` | Connection object method.  Creates cursor object, which is the equivalent of a file handle.
`.execute()` | Cursor object method.  Runs/executes SQL code specified as a parameter in quotes
`.executescript()` | Same as `execute()` except we can run multiple SQL commands separated by ";" at once.
`.fetchall()` | Cursor object method.  Returns (brings into Python) all the records from a RDBMS table that are currently selected.  Returns a list of records.  Each record is a tuple.  If nothing selected returns an empty list.
`.fetchone()` | Cursor object method.  Same as `fetchall()` but only returns first record.  If nothing selected returns None.
`.commit()` | Connection object method.  Forces data to be written to database file on hard drive.  Up until then, all info is in RAM.
`.close()` | Connection object method.  Closes SQLite database file
`?` | Is a placeholder used within the SQL code.  It helps prevent SQL injection.  The ? is replaced by a value in a tuple.  Multiple ? can be used.

---

**EXAMPLES**

In [4]:
import sqlite3
import re

**Simple Example**

In [5]:
con = sqlite3.connect('output/my_db.db')
print(type(con))

cur = con.cursor()
print(type(cur))

# Deletes table if it already exists.
# We do this, so we can run the table creation code multiple times
# If there were already a table with that name in the db we would get an error
cur.execute('DROP TABLE IF EXISTS Breakfast')

cur.execute('CREATE TABLE Breakfast (column_food TEXT (128))')

cur.executescript('''
    INSERT INTO Breakfast (column_food) VALUES ("Spam");
    INSERT INTO Breakfast (column_food) VALUES ("Beans")
''')

con.commit()

# SELECT certain records in table to work with
cur.execute('SELECT * FROM Breakfast WHERE column_food = "Spam"')
for record in cur:  # Only looks at selected records
    print(type(record))
    print(record)

con.close()

<class 'sqlite3.Connection'>
<class 'sqlite3.Cursor'>
<class 'tuple'>
('Spam',)


**Complex Example**
- Create table with the total number of times a domain is seen in an email that was sent in the mbox file
    - E.g. the domain of "lumberjack@humboldt.edu" is "humboldt.edu"
- We want to count how many times this domain was used as a sender

In [6]:
con = sqlite3.connect('output/emaildb.sqlite')
cur = con.cursor()

# Drop table if exists so that we don't repeat counting steps if we run code multiple times
cur.execute('DROP TABLE IF EXISTS table_counts')

# Create new table
# Instead of a Python dictionary we are using a SQLite table
# Our key is now column_domain 
# Our value is now column_count
cur.execute('CREATE TABLE table_counts (column_domain TEXT, column_count INTEGER)')

# Sender emails start with 'From '. 
# Used so we don't create blank lists in next step
# Isolate domain one line at a time
fh_mbox = open('input/mbox.txt', 'rt')
for s_line in fh_mbox:
    if s_line.startswith('From '):
        l_domain = re.findall('^From \S+@(\S+)', s_line)
        s_domain = l_domain[0]
        
        # The following chunks of code are similar to the .get method for dicionaries
        # For each domain name we find, our goal is to add one to the record's value in the column_count
        # Select a record that has a value of the current domain
        # The ? is a placeholder meant to stop SQL injection
        # The portion after the comma is a tuple with one value in it.  The tuple will replace the ?
        # Using ? is not necessary for us here, but good practice
        cur.execute('SELECT column_count FROM table_counts WHERE column_domain = ?', (s_domain,))
      
        # Fetchone pulls first currently selected record from table into Python
        # So the record with the current domain is returned
        record = cur.fetchone()
       
       # The first time a domain is seen there is no record yet
       # We insert a new record withthe domain and a count of 1
        if record is None:
            cur.execute('''
                INSERT INTO table_counts (column_domain, column_count)
                VALUES (?,1)''', (s_domain,))
        
        # If the domain is already in the table then we add one to the count
        else:
            cur.execute('''
                UPDATE table_counts
                SET column_count = column_count + 1
                WHERE column_domain = ?''', (s_domain,))
        
        con.commit()

# Select 10 most popular domains
cur.execute('''
    SELECT *
    FROM table_counts
    ORDER BY column_count DESC
    LIMIT 10--00'
''')

# Make selection a variable using fetchall.
# This gives us a list of records
# Each record is a tuple
l_top_10_domains = cur.fetchall()
print(type(l_top_10_domains))
for record in l_top_10_domains:
    print(record)

con.close()

<class 'list'>
('iupui.edu', 536)
('umich.edu', 491)
('indiana.edu', 178)
('caret.cam.ac.uk', 157)
('vt.edu', 110)
('uct.ac.za', 96)
('media.berkeley.edu', 56)
('ufp.pt', 28)
('gmail.com', 25)
('et.gatech.edu', 17)


---

### MS Excel

- Tabular data is often handled with the `pandas` library.  `pandas` can read and write Excel files.  However, there may be times when we need to automate the creation of Excel files.
- `openpyxl` is the most popular module for working with Excel documents
-  An Excel document has 3 main parts:
    1. **Workbook**--entire Excel document
    1. **Sheet/worksheet**--single page of the workbook
    1. **Cells**-- cells that can be described by their column letter and row number
- `openpyxl` has 3 main objects:
    1. **Workbook object**
    1. **Sheet object**--note that sheet indices start at 0 like they do in Python sequences.
    1. **Cell object**--note that cell indices start at 1 like the they do in Excel.

- Code for working with workbook objects:

Code | Use
--- | ---
`openpyxl` | Module
`openpyxl.Workbook()` | Returns workbook object by creating new workbook.  Starts with one sheet named "Sheet".
`openpyxl.load_workbook(<FILENAME.xlsx>)` | Returns workbook object by opening existing workbook.   If we want to retrieve the result of a cell’s formula instead of the cell's formula itself, use argument `data_only = True`.
`.sheetnames` | Workbook object attribute.  Returns list of sheet names.
`.create_sheet()` | Workbook object method.  Create sheet. By default names sheet `Sheet#` where number is one more than any other existing sheet name.   Optionally, specify title with argument `title = "<SHEETNAME>"`. By default places sheet after existing sheets.  Optionally, specify index with argument `index = #`.
`["<SHEETNAME>"]` | Used upon workbook object.  Returns specified sheet object.
`.save("<FILENAME>.xlsx")` | Workbook object method. Save workbook.  If we open an existing workbook and edit it, we should always save to a new file name in case there are errors in code.

- Code for working with sheet objects:

Code | Use
--- | ---
`.title` | Sheet object attribute.  Returns sheet name as string.
`.title = "<SHEETNAME>"` | Change sheet name
`openpyxl.utils.column_index_from_string()` | Return converted column letter as integer
`openpyxl.utils.get_column_letter()` | Return converted column integer as letter

- Code for accessing cell objects

Code | Use
--- | ---
`["<LETTER_NUMBER>"]` | Used upon sheet object.  Returns specified cell object.
`["<LETTER_NUMBER>"] = <VALUE>` | Change cell value.  New value Python data type automatically converted into correct Excel data type.  Value can be a string with an Excel formula in it.
`.cell(row = #, column = #)` | Sheet object method. Returns specified cell object. Column argument uses integer instead of letter.  Optionally, also specify cell value with argument `value = <VALUE>` or by `.cell().value = <VALUE>`.  Alternative to `["<LETTER_NUMBER>"]`.
`["<LETTER_NUMBER>":"<LETTER_NUMBER>"]` | Used upon sheet object.  Returns tuple of tuples of cell objects.  The inner tuple is a row of cell objects.  The outer tuple contains the row tuples.  There are multiple ways to get cell objects grouped  by row, such as `tuple(<SHEET_OBJECT>.rows)`.  There are also ways to get cell objects grouped by column, such as `tuple(<SHEET_OBJECT>.columns)`.

- Code for working with cell objects

Code | Use
--- | ---
`.value` | Cell object attribute.  Returns the cell's value.  Automatically converts Excel data type to correct Python data type.

---

**EXAMPLES**

In [7]:
import openpyxl

**3 Objects**

In [8]:
wo = openpyxl.load_workbook('input/example.xlsx')
print(type(wo))
so = wo['Sheet1']
print(type(so))
co_b1 = so['B1']
co_b1 = so.cell(row = 1, column = 2) # Does same thing
print(type(co_b1))

<class 'openpyxl.workbook.workbook.Workbook'>
<class 'openpyxl.worksheet.worksheet.Worksheet'>
<class 'openpyxl.cell.cell.Cell'>


**Write New Workbook, Sheet, and Cell Values**

In [9]:
wo = openpyxl.Workbook()
wo.create_sheet(title = 'good_sheet', index = 1)
so['A1'] = 'Column A'
so['B1'] = 'Column B'
so['A2'] = "Spam"
so['B2'] = "Eggs"
wo.save('output/example_new.xlsx')

---

### Google Sheets

- Tabular data is often handled with the `pandas` library. However, there may be times when we want to automate downloads from Google Sheets or uploads to Google Sheets.
- Google Sheets is analogous to MS Excel, but on the cloud.  Working with Google Sheets is more complicated than Excel because it is cloud based and attached to a Google account at all times.  There is a Google API that tells Python scripts how to interact with it.  We can avoid some of the more complicated parts of the API by using a third party Python library.
- `gspread` is the most popular Python library for working with Google Sheets.  We'll focus on linking the Python script to Google Sheets (authentication) and provide some general tips for using `gspread`.
- We'll provide a single code example, but this will not be functional without a valid credential.

**AUTHENTICATION**

Before we an connect to a Google Sheets spreadsheet we must authenticate our Python script.  The `gspread` documentation notes that using Google Colaboratory (Colab) may be the easiest route as Colab is integrated with the Google Drive.  This can make the authentication of accounts  easier.  If we don't want to use Colab we can take the following steps (as of 2021):

1. Log into a Google account
1. Navigate to the Google Developer Console and create a Google Cloud Platform project
1. Enable both Google Drive and Google Sheets APIs for this project
1. Create an "OAuth consent screen" for the "app" we are making
    - Publishing status = Testing
    - User Type = External (Internal not an option)
    - Test Users = add Google account email
1. Create Credentials for OAuth 2.0 Client ID for Desktop App
1. Download this JSON credential file and rename it `credentials.json`
1. For Windows, place credentials file in path: `C:/Users/<USER>/AppData/Roaming/gspread/credentials.json.` An `authorized_user.json` file will be created in this folder automatically after the first Python script with `gspread` is run.  
    - Note that the gspread folder will need to be created. `gspread` documentation tells us to place this in `AppData`, but this raised an exception that hinted we needed to place the file in the`Roaming` folder.
1. *Do not share credentials with anyone.  They do not grant access to a Google account, but do grant access to the spreadsheet.*

**TIPS**

- A Google Sheets document has 3 main parts:
    1. **Spreadsheet**--equivalent of an Excel **workbook**
    1. **Sheet**
    1. **Cell**
- `gspread` has 4 main objects:
    1. **Client object**-authenticates Google account
    1. **Spreadsheet object**
    1. **Worksheet object**--represents a Google sheet.  Worksheet indices start at 0 like they do in Python sequences
    1. **Cell object**--cell indices start at 1 like the they do in Excel and Google Sheets.
- A sheet can have up to 5 million cells. However, the fewer the faster.  It is a good idea to set the column and row count of a sheet to only those needed.
- Updating (changing) cells individually takes about 1 second per cell.  This is because each time a method updates an individual cell, that method sends an API request.  It is faster to modify all cells in entire sheet in one API request.
- Google Sheets sets quotas for read and write operations.   The quota is 100 requests per 100 seconds per user. If the script hits that limit, `APIError 429 RESOURCE_EXHAUSTED` is raised.
- To increase speed and avoid quota exceptions:
    1. Download all cell values from a sheet with one method call
    1. Modify values with Python. The `pandas` Python library is often used.
    1. Upload modified cell values to Google Sheets with one method call

- Code for working with client objects

Code | Use
--- | ---
`gspread` | Module
`gc = gspread.oauth()` | Returns a client object.  Assigned to variable `gc` in documentation.  The first time this is run, it should launch browser window(s) where Google asks for verification.
`.create("<TITLE>")` | Client object method.  Return spreadsheet object by creating new spreadsheet.
`.open("<TITLE>")` | Client object method.  Return spreadsheet object by accessing existing Google Sheet spreadsheet with title.  Will cause error if multiple spreadsheets have same title.  Best to use key or URL.
`.open_by_url("<URL>")` | Client object method.  Return spreadsheet object by accessing existing Google Sheet spreadsheet using spreadsheet URL.
`.open_by_key("<KEY>")` | Client object method.  Return spreadsheet object by accessing existing Google Sheet spreadsheet.    Key found in URL, after the `spreadsheet/d/` part and before the `/edit` part.

- Code for working with spreadsheet objects

Code | Use
--- | ---
`.add_worksheet()` | Spreadsheet object method.  Returns worksheet object by creating new worksheet.  Arguments are `title = "<TITLE>"`, `rows = "#"`, and `cols = "#"`.
`.worksheets()` | Spreadsheet object method.  Return list of all worksheets.
`.worksheet("<TITLE>")` | Spreadsheet object method.  Returns worksheet object.
`.get_worksheet(<INDEX #>)` | Spreadsheet object method.  Returns worksheet object.

- Code for working with worksheet objects and cell objects

Code | Use
--- | ---
`.acell("<LETTER_NUMBER">)` | Worksheet object method.  Returns cell object.  Optionally, return formula instead of cell value using argument `value_render_option='FORMULA'`.
`.cell(row = #, col = #)` |  Worksheet object method.  Returns cell object.  Optionally, return formula instead of cell value using argument `value_render_option='FORMULA'`.
`pd.DataFrame(<WORKSHEET_OBJECT>.get_all_records())` | Convert a worksheet object into Pandas data frame object.  Pandas is imported as `pd`.
`<WORKSHEET_OBJECT>.update([<DF>.columns.values.tolist()] + <DF>.values.tolist())` | Convert Pandas data frame object into a worksheet object

---

**EXAMPLES**

**4 Objects**

- The Python code in this cell raises an exception when run
- To avoid exceptions and allow the ipython kernel to run all code cells at once, this cell has been converted into a Markdown cell
- Markdown grammar displays text as if it were Python code
- It raises an exception because the token in the credential file we have generated and use to log in expired

```Python
import gspread

gc = gspread.oauth()
print(type(gc))
spreadsheet_object = gc.open_by_key('<KEY>')
print(type(spreadsheet_object))
worksheet_object = spreadsheet_object.get_worksheet(0)
print(type(worksheet_object))
cell_object = worksheet_object.cell(row = 2, col = 2)
print(type(cell_object))
```

---

### PDFs
- **PDF**--portable document format.  File extension is .pdf.  Created by Adobe in 1992.  In addition to text, they store font, color, layout information, images, and sometimes even JavaScript.  PDFs are binary files, which makes them much more complex than plaintext files.
- There are additional PDF standards for specific use cases like PDF/A, PDF/E, PDF/X, PDF/UA, and PDF/VT
- There are different types of functionality when it comes to PDF handling programs:
    1. **Editors**--edit or otherwise modify PDF files
        - E.g. Adobe Acrobat Pro, Nitro Pro, Foxit PDF Editor, etc.
    1. **Creators**--convert other file formats to PDFs
        - E.g. any PDF editor, Adobe Illustrator + Photoshop, GIMP,  MS + Libre + Google office suites, LaTeX/Tex programs, etc.
        - E.g. some PDF creator programs like photo editors and office suites also allow us to perform PDF editing indirectly.  E.g. convert PDF to Word document, edit Word Document, then export back to PDF.
    1. **Readers**--open, read, and print PDFs
        - E.g. any PDF editor, Adobe Reader, Nitro PDF Reader, Foxit Reader, all web browsers, etc.
    1. **Converters**--convert PDF files to other formats
        - E.g. any PDF editor, software libraries like poppler and MuPDF, many websites, etc.
- When it comes to text, we are concerned with 3 types of PDFs:
    1. **Image only PDF**--PDF of an image.  Single image layer.  Can not copy text nor search text.
    1. **Image + OCR (Searchable) PDF**--when an image only PDF (or other type of image file) is processed with **optical character recognition (OCR)** algorithms.  OCR recognizes text and places it in a new layer.  Can then copy and search for text.  Works well on documents that have all text  Can perform poorly on documents that have more styling or pictures.
    1. **Digitally Created (True PDF)**--PDF created from original document (like .docx file).  Easy to copy and search for text.
- There does not appear to be a clear favorite, but the most promising libraries in 2021 include:
    1. **PyMuPDF**--PDF text and image extraction, PDF encryption/decryption, and PDF editing at the page and document level.  E.g. we could not add a new sentence to page 4 of a PDF; however, we could select page 4 and write it to a new PDF file.  Optionally uses Python-tesseract if OCR is needed.
    1. **Python-tesseract**--Google library for OCR text extraction
    1. **Camelot**--PDF table text extraction into Pandas dataframe
- We won't provide any example code, but know that these libraries exist if needed.  

---

### MS Word
- **Word document**--file extension is .docx.  Binary file. In addition to text, Word documents store font, color, layout information, images, etc.
- We can add new paragraphs, headings, breaks, and pictures to the Word document, though only to the end of a Word document.  Though easier to work with than PDFs, Word documents are still hard to manipulate with Python.
- We'll use the `docx` module to work with Word documents.  Note that the library is named `python-docx` on PyPI and Conda Forge.  Do NOT download the package named `docx`.  With that noted, we'll be referring to the `python-docx` download as`docx` from now on as that is the module that we import into Python scripts.
- The `docx` module breaks a Word document into objects.  The 3 main ones are:
    1. **Document object**--the entire Word document
    1. **Paragraph object**--paragraphs are designated in Word with Enter or Return while typing
    1. **Run object**--contiguous run of text with the same style.  A style is a collection of text attributes in Word like font, size, and color.

![](images/word_docx_runs.jpg)

Code | Use
--- | ---
`docx` | Module
`docx.Document()` | Return document object by creating new document.  Optionally, load existing document by passing `"<FILENAME>"` as argument.
`.save("<FILENAME>")` | Document object method.  Write file.
`.add_paragraph("<PARAGRAPH_TEXT>")` | Document object method.  Adds paragraph.
`.paragraphs` | Document object attribute.  Returns list of paragraph objects. Use `[<INDEX #>]` on list to return specified paragraph object.
`.add_run("<TEXT>")` | Paragraph object method.  Add new run.
`.runs` | Paragraph object attribute.  Returns list of runs.
`.text` | Paragraph object or run object attribute.  Return text. Use ` = "<TEXT>"` to change text.

---

**EXAMPES**

In [10]:
import docx

**3 Main Objects**

In [11]:
do = docx.Document()
print(type(do))
do.add_paragraph("Hello world")
po = do.paragraphs[0]
print(type(po))
ro = po.runs[0]
print(type(ro))

<class 'docx.document.Document'>
<class 'docx.text.paragraph.Paragraph'>
<class 'docx.text.run.Run'>


**Create and Write**

In [12]:
do = docx.Document()
do.add_paragraph("")
po = do.paragraphs[0]
ro = po.add_run("Hello world")
ro.text = "Hello moon"
do.save("output/example.docx")

---

### Images

- There are many photo editor programs out there, from Adobe Photoshop to GIMP
- There are also many Python libraries that can be used to handle image files.  Some of the most popular image handling libraries are actually aimed at machine learning.  These include NumPy, SciPy, Scikit-image, and OpenCV-Python.
- The pillow library is an active, popular fork of the no longer maintained PIL library.  PIL stands for Python Imaging Library.  Unlike the machine learning libraries listed above, pillow excels at batch processing, image handling, and basic photo editing.
- If each photo we are working with is unique, then other photo editor programs will likely be more useful
- If we have to perform repetitive, simple image handling, then pillow may be a useful alternative to programs like Adobe Photoshop and GIMP (which also have batch processing capabilities)
- We'll provide some background information on image files and then use a couple basic functions from the pillow library.  There is much more functionality that can be found in online documentation.
- **Subtractive color model**--CMYK (cyan, magenta, yellow, and black) are printed on a white background as a series of dots.  Used for print documents. 
- **Additive color model**--displays red, green, and blue (RGB) light over a black background.  Mixing these light colors creates new ones.  Additive color model used in computer and television screens.  Each pixel (picture element) has an RGB value.  Can also include an alpha value, as in RGBA.
- **Alpha**--transparency level.  Alpha is often a value between 0 and 1, with 0 being transparent and 1 being opaque.
- In pillow, each color (and alpha) in each pixel takes on a value ranging from 0-255.  These values are stored in a tuple: (R, G, B, A).  E.g. red is (255, 0, 0, 255).
- In pillow, there are many color names like "red" or "cornflower blue" with pre-determined tuples.  These can be found in documentation.  A handful of common ones are:
    - Red = (255, 0, 0, 255)
    - Green = (0, 128, 0, 255)
    - Blue = (0, 0, 255, 255)
    - White = (255, 255, 255, 255)
    - Black = (0, 0, 0, 0)
- Image pixels are arranged in a grid and addressed with x and y coordinates.  Unlike the normal Cartesian grid, the origin for image pixels is in the top left.  X values increase as one moves right, but y values increase as one moves down!

![](images/pixel_coordinates.jpg)

- A single pixel can be specified using a tuple of two integers, (x, y).
- The size of the entire image can be specified with a tuple or two integers (width, height).
- A "box" tuple can specified a 2D range of pixels using four integers.  In order, these are:
    1. Left.  x coordinate of leftmost edge of box
    1. Top.  y coordinate of uppermost edge of box
    1. Right.  x coordinate of one pixel to the right of the rightmost edge of box
    1. Bottom.  y coordinate of one pixel lower than the bottom edge of the box.

![](images/pixel_box_coordinates.jpg)

Code | Use
--- | ---
`from PIL import Image` | Main module.  Even though this is the pillow module, the creators maintained backwards compatibility with PIL, which lead to this import grammar.
`Image.open('<FILENAME>')` | Return image object from specified file
`.size` | Image object attribute.  Return size as tuple of two integers.
`.filename` | Image object attribute.  Return image filename.
`.format` | Image object attribute.  Return image format.
`.format_description` | Image object attribute.  Return image format description.
`.crop(<BOX_TUPLE>)` | Image object method.  Creates copy of image object then crops copy so new image object has specified 2D pixel range.  Does not modify original image object.
`.paste(<IMAGE_OBJECT>, (x,y))` | Image object method.  Creates copy of image object and then pastes specified image object on top of that.  The top left corner of the second image will be located at the (x,y) coordinates specified in the first image.
`.save('<FILNAME>')` | Image object method.  Save image with specified filename and file format indicated by extension.  Pillow converts between formats for us.
`from PIL import ImageColor` | Module for working with RGBA color
`ImageColor.getcolor("<COLOR>", "RGBA")` | Return tuple of RGBA values for specified color

---

**EXAMPLES**

In [13]:
from PIL import Image

**Change File Type**

In [14]:
jpg_image = Image.open("input/monty_python.jpg")
print(type(jpg_image))
jpg_image.save("output/monty_python.png")
png_image = Image.open("output/monty_python.png")
print(type(png_image))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>


**`.crop()` and `.paste()`**

In [15]:
rabbit_image = Image.open("input/rabbit.jpg")
t_box = (130, 200, 400, 750)
rabbit_image = rabbit_image.crop(t_box)
monty_image = Image.open("input/monty_python.jpg")
monty_image.paste(rabbit_image, (980, 1300))
monty_image.save("output/monty_python_rabbit.jpg")

---